Ebook: A Hands-On Guide to Biomechanics Data Analysis with Python and AI
Author: Dr. Hossein Mokhtarzadeh
Powered by PoseIQ™

This notebook loads sample biomechanics data and shows how to bring it into Python.
Click Runtime → Run all.

In [ ]:
# Chapter 5 bridge, reuse Chapter 4 state if available

import os, json, base64, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 0) Colab one-time setup only if c3d is missing
try:
    _ = c3d["data"]["points"]
    print("c3d found from previous chapter.")
except Exception:
    print("c3d not found, running minimal setup used earlier.")
    %pip -q install ezc3d pandas
    !wget -q https://raw.githubusercontent.com/hmok/BiomechPythonAI_Guide/main/notebooks/Chapter1Input.py -O Chapter1Input.py
    !wget -q https://raw.githubusercontent.com/hmok/BiomechPythonAI_Guide/main/notebooks/Chapter2Input.py -O Chapter2Input.py
    %run Chapter1Input.py
    %run Chapter2Input.py

P = c3d["parameters"]

def pval(g,k, default=None):
    try:
        return P[g][k]["value"]
    except Exception:
        return default

def get_rates():
    pt_rate = float(pval("POINT","RATE",[np.nan])[0]) if pval("POINT","RATE",None) is not None else np.nan
    an_rate = float(pval("ANALOG","RATE",[np.nan])[0]) if pval("ANALOG","RATE",None) is not None else np.nan
    return pt_rate, an_rate

def reshape_analogs():
    raw = c3d["data"].get("analogs", None)
    if raw is None:
        return np.zeros((0,0))
    A = np.array(raw)
    if A.ndim == 2:
        return A
    if A.ndim == 3:
        if A.shape[0] < 16 and A.shape[1] > A.shape[0]:
            return np.moveaxis(A, 0, -1).reshape(A.shape[1], -1)
        return A.reshape(A.shape[0], -1)
    return A.reshape(A.shape[0], -1)

def get_labels():
    analog_labels = pval("ANALOG","LABELS", []) or []
    return [str(x) for x in analog_labels]

def soft_detrend(y): return y - np.nanmedian(y)
def ensure_positive_down(y):
    pos_peak = np.nanpercentile(np.maximum(y, 0), 99)
    neg_peak = np.nanpercentile(np.maximum(-y, 0), 99)
    return y if pos_peak >= neg_peak else -y

def pick_fz_or_mag(an, labels):
    fz_idx = [i for i,s in enumerate(labels) if "fz" in s.lower() or "vf" in s.lower()]
    for i in fz_idx:
        y = an[i].astype(float)
        y = ensure_positive_down(soft_detrend(y))
        if np.nanmax(np.abs(y)) > 1e-6:
            return y, "Fz", i
    fx = next((i for i,s in enumerate(labels) if "fx" in s.lower()), None)
    fy = next((i for i,s in enumerate(labels) if "fy" in s.lower()), None)
    fz = next((i for i,s in enumerate(labels) if "fz" in s.lower()), None)
    if fx is not None and fy is not None and fz is not None:
        mag = np.sqrt(an[fx]**2 + an[fy]**2 + an[fz]**2).astype(float)
        return soft_detrend(mag), "GRF magnitude", (fx,fy,fz)
    return None, None, None

def clean_contact_bool(x, min_len_samples):
    x = x.astype(bool)
    on  = np.where(np.diff(x.astype(int)) == 1)[0] + 1
    off = np.where(np.diff(x.astype(int)) == -1)[0] + 1
    if x[0]:  on  = np.r_[0, on]
    if x[-1]: off = np.r_[off, len(x)]
    y = x.copy()
    for s,e in zip(on,off):
        if e - s < min_len_samples:
            y[s:e] = False
    return y

def detect_stance_pairs(sig, an_rate, thr_frac=0.05, thr_abs_N=20.0, min_stance_s=0.15):
    peak_est = np.nanpercentile(np.abs(sig), 98)
    thr = max(thr_frac * peak_est, thr_abs_N)
    contact = sig > thr
    contact = clean_contact_bool(contact, int(min_stance_s * an_rate))
    starts = np.where(np.diff(contact.astype(int)) == 1)[0] + 1
    ends   = np.where(np.diff(contact.astype(int)) == -1)[0] + 1
    if contact[0] and (len(ends) == len(starts) + 1): starts = np.r_[0, starts]
    if contact[-1] and (len(starts) == len(ends) + 1): ends = np.r_[ends, len(contact)]
    return [(s,e) for s,e in zip(starts,ends) if e > s]

def resample_cycle(y, s, e, n=101):
    if e - s < 3:
        return np.full(n, np.nan)
    xs = np.linspace(s, e - 1, e - s)
    xt = np.linspace(s, e - 1, n)
    return np.interp(xt, xs, y[s:e])

# 1) Rebuild or reuse the Chapter 4 state
pt_rate, an_rate = get_rates()
an = reshape_analogs()
analog_labels = get_labels()

try:
    sig  # from Chapter 4
    pairs
    print("Using vertical force and stance pairs from Chapter 4.")
except NameError:
    sig, sig_name, src = pick_fz_or_mag(an, analog_labels)
    if sig is None or np.isnan(an_rate):
        raise RuntimeError("Could not build a reporting signal. Check analog labels and ANALOG.RATE.")
    pairs = detect_stance_pairs(sig, an_rate)
    print(f"Detected {len(pairs)} stance phases using {sig_name} from this chapter.")

t_an = np.arange(an.shape[1]) / an_rate
print("Bridge ready. Proceed to metrics and export.")